In [1]:
import selenium

In [2]:
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
path = ChromeDriverManager().install()

In [4]:
import pandas as pd

In [5]:
df = pd.read_excel("Company_name.xlsx")
df.head()

,Company name
0,"Kyndryl Holdings, Inc."
1,"Keysight Technologies, Inc."
2,Kirby Corporation
3,"Kinder Morgan, Inc."
4,OPENLANE Inc


In [6]:
df[df['Company name']==	'Denali Therapeutics Inc.']

,Company name
1202,Denali Therapeutics Inc.


In [7]:
df2 = df.iloc[1202:1302]

In [36]:
df.columns

Index(['Company name'], dtype='object')

In [22]:
df[df['Company name']=='Energizer Holdings, Inc.']

,Company name
642,"Energizer Holdings, Inc."


In [8]:
company_list = df2.values

In [9]:
len(company_list)

100

In [10]:
company_list[0]

array(['Denali Therapeutics Inc.'], dtype=object)

In [10]:
company_name = df2['Company name'].values

In [ ]:
import csv
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

website = 'https://finance.yahoo.com/'

output_file = "company_revenue_data_yahoo3.csv"

service = Service(executable_path=path)
driver = webdriver.Chrome(service=service)
output_data = []

# ==== Loop through each company ====
for company in company_list:
    try:
        driver.get(website)
        search_container = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//input[@class='_yb_d7nywr _yb_owcm9n  _yb_ikgg1k finsrch-inpt  ']"))
        )
        search_container.click()
        search_container.send_keys(company)
        time.sleep(2)

        for i in range(3):
            matched_name = np.nan
            revenue_ttm = np.nan

            try:
                result_xpath = f"//ul[@role='listbox']/li[contains(@data-id , 'result-quotes-{i}')]/div[@class='modules-module_quoteLeftCol__yV9LA modules-module_Ell__N1WPm modules-module_IbBox__NPYId']"
                search_result = WebDriverWait(driver, 4).until(
                    EC.element_to_be_clickable((By.XPATH, result_xpath))
                )
                search_result.click()
                time.sleep(3)

                try:
                    matched_name = WebDriverWait(driver, 4).until(
                        EC.presence_of_element_located((By.XPATH, "//h1[@class='yf-xxbei9']"))
                    ).text.strip()
                except Exception:
                    matched_name = np.nan

                try:
                    stats_list = WebDriverWait(driver, 5).until(
                        EC.presence_of_all_elements_located((By.XPATH, "//p[@class='value yf-lc8fp0']"))
                    )
                    revenue_ttm = stats_list[3].text.strip() if len(stats_list) > 3 else np.nan
                except Exception:
                    revenue_ttm = np.nan

                print(f"\n✅ {company} → {matched_name} | Revenue (TTM): {revenue_ttm}")

            except Exception as e:
                print(f"❌ Skipped result-{i+1} for '{company}': {e}")

            output_data.append({
                'Input_Company': company,
                'Matched_Company': matched_name,
                'Revenue_TTM': revenue_ttm
            })

            # Go back to search
            driver.get(website)
            search_container = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//input[@class='_yb_d7nywr _yb_owcm9n  _yb_ikgg1k finsrch-inpt  ']"))
            )
            search_container.click()
            search_container.send_keys(company)
            time.sleep(2)

    except Exception as e:
        print(f"❌ Error while processing '{company}': {e}")

driver.quit()

# ==== Save to CSV ====
df = pd.DataFrame(output_data)
df.to_csv(output_file, index=False)

print(f"\n📁 CSV saved as: {output_file}")



✅ ['Denali Therapeutics Inc.'] → Denali Therapeutics Inc. (DNLI) | Revenue (TTM): --

✅ ['Denali Therapeutics Inc.'] → Denali Therapeutics Inc. (4DN.F) | Revenue (TTM): --

✅ ['Denali Therapeutics Inc.'] → Denali Therapeutics Inc. (D2NL34.SA) | Revenue (TTM): --

✅ ['United Rentals, Inc.'] → United Rentals, Inc. (URI) | Revenue (TTM): 15.35B
❌ Skipped result-2 for '['United Rentals, Inc.']': Message: 
Stacktrace:
	GetHandleVerifier [0x010680E3+60707]
	GetHandleVerifier [0x01068124+60772]
	(No symbol) [0x00E90683]
	(No symbol) [0x00ED8660]
	(No symbol) [0x00ED89FB]
	(No symbol) [0x00F21022]
	(No symbol) [0x00EFD094]
	(No symbol) [0x00F1E824]
	(No symbol) [0x00EFCE46]
	(No symbol) [0x00ECC5D3]
	(No symbol) [0x00ECD424]
	GetHandleVerifier [0x012ABBC3+2435075]
	GetHandleVerifier [0x012A7163+2416035]
	GetHandleVerifier [0x012C350C+2531660]
	GetHandleVerifier [0x0107F1B5+155125]
	GetHandleVerifier [0x01085B5D+182173]
	GetHandleVerifier [0x0106F9B8+91640]
	GetHandleVerifier [0x0106FB60+92064

In [31]:
df = pd.DataFrame(output_data)
df.to_csv(output_file , index = False)

In [32]:
len(df)

437